### Testowanie uczenia się DFA z pomocą systemu przepisywania więzów

In [48]:
import copy, sys
import cProfile

sys.path.append("../")
sys.path.append("../inferring")
sys.path.append("../utils")
sys.path.append("../utils/DFA")
from importlib import reload


import inferring.Inferring as Inferring, inferring.InferringDFA as InferringDFA
import utils.automats.DFA, utils.automats.DFA.PatternDFA as PatternDFA
import utils.oracles.SRS as SRS, utils.oracles.SRSconv as SRSconv
reload(SRS)
reload(SRSconv)

reload(Inferring)
reload(InferringDFA)

reload(utils.automats.DFA)
reload(PatternDFA)

from inferring.Inferring import Inferring
from utils.automats.DFA.DFA import DFA
from utils.automats.DFA.PatternDFA import PatternDFA
from utils.oracles.SRSconv import SRSconv

from inferring.InferringDFA import InferringDFA

In [49]:
def run_learning_process(test, oracle=None, check_consistency=False, equiv_query_fashion="BFS", debug=False):
    _dfa = copy.deepcopy(test.dfa)
    learn_dfa = InferringDFA(_dfa, 
                                oracle, 
                                check_consistency=check_consistency, 
                                equiv_query_fashion=equiv_query_fashion,
                                debug=debug)
        
    dfa, cnt, cnt_ex = learn_dfa.run(counterexamples=True)
    print(f"kontrprzykłady: {cnt_ex}, cnt = {cnt}")
    return copy.deepcopy(dfa) 

In [50]:
class test_example():
    def __init__(self, type, dfa, descrip=''):
        self.type = type
        self.dfa = copy.deepcopy(dfa)
        self.descrip = descrip

In [51]:
test_examples = []

dfa_ = DFA()
dfa_.create_convolution(dfa1 = DFA(Q=4, input_signs=['a', 'b'], δ = {(0, 'a'):1, (0, 'b'):0, (1, 'a'):1, (1, 'b'):2, (2, 'a'):3, (2, 'b'):0, (3, 'a'):3, (3, 'b'):3}, F=set([3])),
                        dfa2 = DFA(Q=4, input_signs=['a', 'b'], δ = {(0, 'a'):0, (0, 'b'):1, (1, 'a'):0, (1, 'b'):2, (2, 'a'):0, (2, 'b'):3, (3, 'a'):3, (3, 'b'):3}, F=set([3])))
test = test_example(type = "conv", dfa = dfa_, descrip = "splot, dfa1 - wzorzec 'aba', dfa2 - wzorzec 'bbb'")
test_examples.append(test)

# result, without SRS at all: cnt = [1388, 7]

dfa_ = DFA()
dfa_.create_convolution(dfa1=DFA(Q=3, input_signs=['a', 'b'], δ={(0,'a'):0, (0,'b'):1, (1, 'a'):0, (1, 'b'):2, (2, 'a'):2, (2, 'b'):2}, F= set([2])), 
                        dfa2=DFA(Q=3, input_signs=['a', 'b'], δ={(0,'a'):1, (0,'b'):0, (1, 'a'):2, (1, 'b'):0, (2, 'a'):2, (2, 'b'):2}, F= set([2])))
test = test_example(type="conv", dfa=dfa_, descrip="splot, dfa1 - wzorzec - 'bb', dfa2 - wzorzec 'aa'")
test_examples.append(test)
# result, without SRS at all: cnt = [289, 3]
    
    # test_example(type="conv", 
    #              dfa = convDFA(type="conv", 
    #                            dfa1=DFA(Q=4, input_signs=['a', 'b'], δ={(0,'a'):0, (0,'b'):1, (1, 'a'):0, (1, 'b'):2, (2, 'a'):0, (2, 'b'):3, (3, 'a'):3, (3, 'b'):3}, F= set([3])),
    #                            dfa2=DFA(Q=4, input_signs=['a', 'b'], δ={(0,'a'):1, (0,'b'):0, (1, 'a'):2, (1, 'b'):0, (2, 'a'):3, (2, 'b'):0, (3, 'a'):3, (3, 'b'):3}, F= set([3]))),
    #              description="splot, dfa1 - wzorzec 'bbb', dfa2 - wzorzec 'aaa'"),
    # # result, without SRS at all: cnt = [800, 3]

    # test_example(type="conv",
    #              dfa=convDFA(type="conv",
    #                          dfa1=DFA(Q=2, input_signs=['a', 'b'], δ = {(0,'a'):1, (0, 'b'):0, (1, 'a'):1, (1, 'b'):1}, F=set([1])),
    #                          dfa2=DFA(Q=2, input_signs=['a', 'b'], δ = {(0,'a'):1, (0, 'b'):0, (1, 'a'):1, (1, 'b'):1}, F=set([1]))),
    #             description="splot, dfa1 - słowa zawierające literke 'a', dfa2 - słowa zawierające literke 'a'"),
    # # result, without SRS at all: cnt = [69, 1]
    # #             
    # test_example(type="conv", 
    #              dfa=convDFA(type="conv",
    #                          dfa1=DFA(Q=4, input_signs=['a', 'b'], δ={(0, 'a'):2, (0,'b'):1, (1, 'b'):1, (1, 'a'):3, (2, 'a'):2, (2, 'b'):3, (3, 'a'):3, (3, 'b'):3}, F=set([3])),
    #                          dfa2=DFA(Q=4, input_signs=['a', 'b'], δ={(0, 'a'):2, (0,'b'):1, (1, 'b'):1, (1, 'a'):3, (2, 'a'):2, (2, 'b'):3, (3, 'a'):3, (3, 'b'):3}, F=set([3]))),
    #             description="splot, dfa1 - słowa zawierające i literę 'a' i literę 'b', dfa2 - słowa zawierające i literę 'a' i literę 'b'"),
    # # result, without SRS at all: cnt = [751, 10]
              
    # test_example(type="dfa", 
    #              dfa=DFA(Q=3, input_signs=['0', '1', '2', '3', '4', '5', '6', '7'], δ={(0,'0'):0, (0,'1'):2, (0,'2'):2, (0,'3'):0, (0,'4'):2, (0,'5'):0, (0,'6'):1, (0,'7'):2, (1,'0'):2, (1,'1'):0, (1,'2'):1, (1,'3'):2, (1,'4'):1, (1,'5'):2, (1,'6'):2, (1,'7'):1, (2,'0'):2, (2,'1'):2, (2,'2'):2, (2,'3'):2, (2,'4'):2, (2,'5'):2, (2,'6'):2, (2,'7'):2}, F=set([0])),
    #              description="zakodowane dodawania w systemie dwójkowym"), 
    # # result, without SRS at all: cnt = [201, 1]

    # test_example(type="conv", 
    #              dfa=convDFA(type="conv", 
    #                          dfa1=DFA(Q=5, input_signs=['a', 'b'], δ={(0,'a'):1, (0,'b'):3, (1,'a'):2, (1,'b'):3, (2,'a'):2, (2,'b'):2, (3,'a'):1, (3,'b'):4, (4,'a'):4, (4,'b'):4}, F=set([2, 4])),
    #                          dfa2=DFA(Q=5, input_signs=['a', 'b'], δ={(0,'a'):1, (0,'b'):3, (1,'a'):2, (1,'b'):3, (2,'a'):2, (2,'b'):2, (3,'a'):1, (3,'b'):4, (4,'a'):4, (4,'b'):4}, F=set([2, 4])),
    #             description="splot, dfa1 - 2 wzorce 'aa' lub 'bb', dfa2 - 2 wzorce 'aa' lub 'bb'")),
    # # result, without SRS at all: cnt = [1437, 7]
                
    # test_example(type="conv", 
    #              dfa=convDFA(type="conv", 
    #                          dfa1=DFA(Q=9, input_signs=['a', 'b'], δ={(0,'a'):1, (0,'b'):3, (1,'a'):2, (1,'b'):3, (2,'a'):2, (2,'b'):5, (3,'a'):1, (3,'b'):4, (4,'a'):7, (4,'b'):4, (5,'a'):5, (5,'b'):6, (6,'a'):6, (6,'b'):6, (7,'a'):8, (7,'b'):7, (8,'a'):8, (8,'b'):8}, F=set([6, 8])),
    #                          dfa2=DFA(Q=9, input_signs=['a', 'b'], δ={(0,'a'):1, (0,'b'):3, (1,'a'):2, (1,'b'):3, (2,'a'):2, (2,'b'):5, (3,'a'):1, (3,'b'):4, (4,'a'):7, (4,'b'):4, (5,'a'):5, (5,'b'):6, (6,'a'):6, (6,'b'):6, (7,'a'):8, (7,'b'):7, (8,'a'):8, (8,'b'):8}, F=set([6, 8]))),
    #             description="splot, dfa1 - 2 wzorce 'aa' i 'bb', dfa2 - 2 wzorce 'aa' i 'bb'"), 
    # # result, without SRS at all: cnt = [20522, 28]

    # test_example(type="conv",
    #                 dfa=convDFA(type="conv",
    #                             dfa1=PatternDFA(input_signs=['a', 'b'], patterns=['aa', 'bb']),
    #                             dfa2=PatternDFA(input_signs=['a', 'b', 'c'], patterns=['aa', 'bb', 'c'])),
    #                 description="splot, dfa1 - 2 wzorce: 'aa', 'bb', dfa2 - 3 wzorce: 'aa', 'bb', 'c'")]
    # # result, without SRS at all: cnt = [121709, 37]


In [52]:
test = test_examples[1] 
print(test.descrip)
test.dfa.print_transitions()

# d = run_learning_process(copy.deepcopy(test), oracle=None, check_consistency=False, equiv_query_fashion="DFS", debug=False)
# d = run_learning_process(copy.deepcopy(test), oracle=SRSconv(), check_consistency=False, equiv_query_fashion="DFS")
d = run_learning_process(copy.deepcopy(test), oracle=SRSconv(), check_consistency=True, equiv_query_fashion="DFS")

d.route("abbaBBAAB")


splot, dfa1 - wzorzec - 'bb', dfa2 - wzorzec 'aa'
(0,a) --> 0
(0,b) --> 3
(0,A) --> 1
(0,B) --> 0
(1,a) --> 1
(1,b) --> 4
(1,A) --> 2
(1,B) --> 0
(2,a) --> 2
(2,b) --> 5
(2,A) --> 2
(2,B) --> 2
(3,a) --> 0
(3,b) --> 6
(3,A) --> 4
(3,B) --> 3
(4,a) --> 1
(4,b) --> 7
(4,A) --> 5
(4,B) --> 3
(5,a) --> 2
(5,b) --> 8
(5,A) --> 5
(5,B) --> 5
(6,a) --> 6
(6,b) --> 6
(6,A) --> 7
(6,B) --> 6
(7,a) --> 7
(7,b) --> 7
(7,A) --> 8
(7,B) --> 6
(8,a) --> 8
(8,b) --> 8
(8,A) --> 8
(8,B) --> 8
stany akceptujące - {8}
kontrprzykłady: [4, 6], cnt = [174, 3]


('abbaBBAAB', 1)

## Przykład dziwny

In [26]:
patterns1 = ['aaa','bba','cca','ddd']
patterns2 = ['aaa','bbb','ccc','ddd']

test_1 = test_example(type="dfa", dfa=PatternDFA(input_signs=['a', 'b', 'c', 'd'], patterns=patterns1, _type="OR"))
test_2 = test_example(type="dfa", dfa=PatternDFA(input_signs=['a', 'b', 'c', 'd'], patterns=patterns2, _type="OR"))
d1 = run_learning_process(test=test_1)
d2 = run_learning_process(test=test_2)

kontrprzykłady: [3, 3, 3, 3], cnt = [289, 5]
kontrprzykłady: [3, 3, 3, 3], cnt = [289, 5]


In [7]:
conv_dfa = convDFA(type="conv", dfa1=d1, dfa2=d2)

In [8]:
d = run_learning_process(test=test_example(type="conv", dfa=conv_dfa), oracle=SRSconv(), check_consistency=True, debug=True)

# aaaAAA
# aaaCCC
# aaaDDD

S = 1, rozmiar E = 9
zamkniętość sprawdzona - S = 1, rozmiar E = 9
hipoteza: 1
(0,a) --> 0
(0,b) --> 0
(0,c) --> 0
(0,d) --> 0
(0,A) --> 0
(0,B) --> 0
(0,C) --> 0
(0,D) --> 0
stany akceptujące - set()
sprawdzam ZGODNOŚĆ z więzami
z niezgodnośći wynikaja takie kontrprzykłady: set()
kontrprzyklad (z zapytania o równoważność)= aaaAAA



S = 1, rozmiar E = 13
zamkniętość sprawdzona - S = 7, rozmiar E = 13
hipoteza: 7
(0,a) --> 1
(0,b) --> 0
(0,c) --> 0
(0,d) --> 0
(0,A) --> 0
(0,B) --> 0
(0,C) --> 0
(0,D) --> 0
(1,a) --> 2
(1,b) --> 0
(1,c) --> 0
(1,d) --> 0
(1,A) --> 1
(1,B) --> 1
(1,C) --> 1
(1,D) --> 1
(2,a) --> 3
(2,b) --> 0
(2,c) --> 0
(2,d) --> 0
(2,A) --> 2
(2,B) --> 2
(2,C) --> 2
(2,D) --> 2
(3,a) --> 3
(3,b) --> 3
(3,c) --> 3
(3,d) --> 3
(3,A) --> 4
(3,B) --> 3
(3,C) --> 3
(3,D) --> 3
(4,a) --> 4
(4,b) --> 4
(4,c) --> 4
(4,d) --> 4
(4,A) --> 5
(4,B) --> 3
(4,C) --> 3
(4,D) --> 3
(5,a) --> 5
(5,b) --> 5
(5,c) --> 5
(5,d) --> 5
(5,A) --> 6
(5,B) --> 3
(5,C) --> 3
(5,D) --> 3
(6,a) -

In [18]:
d.route("bbbBBaB")

('bbbBBaB', 1)

### Duże automaty - ropoznające kilkadziesiąt wzorców (docelowo)

In [5]:
import random 
def random_patterns(input_signs, n=5, max_length=10):
    patterns = []
    for _ in range(n):
        p = ''.join(random.choices(population=input_signs,k=random.randint(1, max_length)))
        patterns.append(p)
    return patterns

random_patterns(input_signs=['a', 'b', 'c', 'd'], max_length=8)

['bbdbcdbd', 'cc', 'ccbdd', 'baba', 'a']

In [ ]:
random.seed(25)

input_signs = ['a', 'b', 'c', 'd']
number_of_patterns = 4
max_pattern_length = 6 

patterns1, patterns2 = random_patterns(input_signs, number_of_patterns, max_pattern_length), random_patterns(input_signs, number_of_patterns, max_pattern_length)

dfa1 = PatternDFA(input_signs=input_signs, patterns=patterns1)
dfa2 = PatternDFA(input_signs=input_signs, patterns=patterns2)

print(f"patterns1 = {patterns1}")
print(f"|Q1| = {dfa1.Q}")

print(f"patterns2 = {patterns2}")
print(f"|Q2| = {dfa2.Q}")

d1 = run_learning_process(test_example(type="dfa", dfa=copy.deepcopy(dfa1)))
d2 = run_learning_process(test_example(type="dfa", dfa=copy.deepcopy(dfa2)))

print(f"after learning |Q1| = {d1.Q}")
s1 = ''.join(patterns1)
ans1 = d1.route(s1)

s2 = ''.join(patterns2)
print(f"after learning |Q2| = {d2.Q}")
ans2 = d2.route(s2)

print(f"{ans1}, {ans2}")

In [8]:
dfa_ = convDFA(type="conv", dfa1=copy.deepcopy(d1), dfa2=copy.deepcopy(d2))

dfa_.route(s1+s2.upper())
print(f"|Q_conv(d1,d2)| = {dfa_.Q}")

test = test_example(type="conv", dfa=dfa_, description=f"splot, dfa1 - wzorce: {patterns1}, dfa2 - wzorce: {patterns2}")
print(test.description)

|Q_conv(d1,d2)| = 1872
splot, dfa1 - wzorce: ['ddad', 'cad', 'aca', 'c'], dfa2 - wzorce: ['dcdcda', 'ccd', 'ccbbcc', 'a']


In [ ]:
cProfile.run('run_learning_process(test_example(type="dfa", dfa=copy.deepcopy(dfa1)))', sort='time')
# cProfile.run('run_learning_process(test, oracle=SRSconv(), check_consistency=True, debug=False)', sort='time')

### Uczenie się splotu DFA - **debugowanie** 

<!-- ### Uczenie się splotów - **debugowanie**  -->